# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda

Vamos trabalhar com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [7]:
import pandas as pd
import seaborn as sns
from seaborn import load_dataset
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

import patsy
import statsmodels.api as sm
import statsmodels.formula.api as smf

%matplotlib inline

In [8]:
df = pd.read_csv('previsao_de_renda.csv')

In [9]:
# Completando a coluna tempo_emprego com a média dos valores da coluna
valor_media = df['tempo_emprego'].mean()
df['tempo_emprego'].fillna(valor_media, inplace=True)

C:\Users\Bruno PC\AppData\Local\Temp\ipykernel_9736\2618388757.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['tempo_emprego'].fillna(valor_media, inplace=True)


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   id_cliente             15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  bool   
 5   posse_de_imovel        15000 non-null  bool   
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  object 
 10  tipo_residencia        15000 non-null  object 
 11  idade                  15000 non-null  int64  
 12  tempo_emprego          15000 non-null  float64
 13  qt_pessoas_residencia  15000 non-null  float64
 14  renda                  15000 non-null  float64
dtypes:

1. Ajuste um modelo para prever log(renda) considerando todas as covariáveis disponíveis.
    - Utilizando os recursos do Patsy, coloque as variáveis qualitativas como *dummies*.
    - Mantenha sempre a categoria mais frequente como casela de referência
    - Avalie os parâmetros e veja se parecem fazer sentido prático.  


2. Remova a variável menos significante e analise:
    - Observe os indicadores que vimos, e avalie se o modelo melhorou ou piorou na sua opinião.
    - Observe os parâmetros e veja se algum se alterou muito.  


3. Siga removendo as variáveis menos significantes, sempre que o *p-value* for menor que 5%. Compare o modelo final com o inicial. Observe os indicadores e conclua se o modelo parece melhor. 
    

In [12]:
# Descobrindo qual categoria é a moda nesse dataframe
colunas = ['sexo', 'posse_de_veiculo', 'posse_de_imovel', 'qtd_filhos', 'tipo_renda', 'educacao','estado_civil', 'tipo_residencia', 'idade', 'tempo_emprego','qt_pessoas_residencia', 'renda']
for coluna in colunas:
    print(df[coluna].value_counts())
    print("\n")

sexo
F    10119
M     4881
Name: count, dtype: int64


posse_de_veiculo
False    9140
True     5860
Name: count, dtype: int64


posse_de_imovel
True     10143
False     4857
Name: count, dtype: int64


qtd_filhos
0     10376
1      3037
2      1376
3       185
4        17
7         5
14        2
5         2
Name: count, dtype: int64


tipo_renda
Assalariado         7633
Empresário          3508
Pensionista         2582
Servidor público    1268
Bolsista               9
Name: count, dtype: int64


educacao
Secundário             8895
Superior completo      5335
Superior incompleto     579
Primário                165
Pós graduação            26
Name: count, dtype: int64


estado_civil
Casado      10534
Solteiro     1798
União        1078
Separado      879
Viúvo         711
Name: count, dtype: int64


tipo_residencia
Casa             13532
Com os pais        675
Governamental      452
Aluguel            194
Estúdio             83
Comunitário         64
Name: count, dtype: int64


idade
40 

In [13]:
# 1) Criando o  modelo sem Treatment
y, X = patsy.dmatrices('np.log(renda) ~ C(sexo) + posse_de_veiculo + posse_de_imovel + qtd_filhos + C(tipo_renda) + C(educacao) + C(estado_civil) + C(tipo_residencia) + idade + tempo_emprego + qt_pessoas_residencia', df)

In [14]:
modelo_0 = sm.OLS(y,X).fit()

modelo_0.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.348
Model:                            OLS   Adj. R-squared:                  0.347
Method:                 Least Squares   F-statistic:                     333.6
Date:                Sat, 23 Nov 2024   Prob (F-statistic):               0.00
Time:                        15:15:00   Log-Likelihood:                -16212.
No. Observations:               15000   AIC:                         3.247e+04
Df Residuals:                   14975   BIC:                         3.266e+04
Df Model:                          24                                         
Covariance Type:            nonrobust                                         
=======================================================================================================
                                          coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
Intercept                               6.6956      0.224     29.905      0.000       6.257       7.134
C(sexo)[T.M]                            0.7947      0.014     57.710      0.000       0.768       0.822
posse_de_veiculo[T.True]                0.0351      0.013      2.693      0.007       0.010       0.061
posse_de_imovel[T.True]                 0.0897      0.013      6.978      0.000       0.064       0.115
C(tipo_renda)[T.Bolsista]               0.2075      0.238      0.870      0.384      -0.260       0.675
C(tipo_renda)[T.Empresário]             0.1543      0.015     10.457      0.000       0.125       0.183
C(tipo_renda)[T.Pensionista]           -0.1910      0.021     -9.123      0.000      -0.232      -0.150
C(tipo_renda)[T.Servidor público]       0.0571      0.022      2.602      0.009       0.014       0.100
C(educacao)[T.Pós graduação]            0.0549      0.151      0.363      0.716      -0.241       0.351
C(educacao)[T.Secundário]              -0.0719      0.056     -1.278      0.201      -0.182       0.038
C(educacao)[T.Superior completo]        0.0479      0.057      0.845      0.398      -0.063       0.159
C(educacao)[T.Superior incompleto]     -0.1024      0.063     -1.614      0.107      -0.227       0.022
C(estado_civil)[T.Separado]             0.2754      0.107      2.568      0.010       0.065       0.486
C(estado_civil)[T.Solteiro]             0.2294      0.105      2.181      0.029       0.023       0.436
C(estado_civil)[T.União]               -0.0382      0.023     -1.662      0.097      -0.083       0.007
C(estado_civil)[T.Viúvo]                0.2855      0.108      2.637      0.008       0.073       0.498
C(tipo_residencia)[T.Casa]             -0.0312      0.052     -0.600      0.548      -0.133       0.071
C(tipo_residencia)[T.Com os pais]      -0.0116      0.058     -0.199      0.842      -0.126       0.103
C(tipo_residencia)[T.Comunitário]      -0.1107      0.103     -1.071      0.284      -0.313       0.092
C(tipo_residencia)[T.Estúdio]           0.0619      0.094      0.659      0.510      -0.122       0.246
C(tipo_residencia)[T.Governamental]    -0.0013      0.062     -0.022      0.983      -0.122       0.120
qtd_filhos                             -0.2364      0.104     -2.262      0.024      -0.441      -0.032
idade                                   0.0051      0.001      6.769      0.000       0.004       0.007
tempo_emprego                           0.0618      0.001     60.439      0.000       0.060       0.064
qt_pessoas_residencia                   0.2605      0.104      2.498      0.013       0.056       0.465
==============================================================================
Omnibus:                        0.716   Durbin-Watson:                   2.021
Prob(Omnibus)

A categoria mais presente nessa análise é tipo_residência : casa com 13532 registros

In [16]:
# Convertendo a coluna para o tipo category
df['tipo_residencia'] = df['tipo_residencia'].astype('category')
# Encontrando o índice
indice_casa = df['tipo_residencia'].cat.categories.get_loc('Casa')
indice_casa

1

In [17]:
# 1) Criando o  modelo usando 1 como Treatment
y, X = patsy.dmatrices('np.log(renda) ~ C(sexo) + posse_de_veiculo + posse_de_imovel + qtd_filhos + C(tipo_renda) + C(educacao) + C(estado_civil) + C(tipo_residencia, Treatment(1)) + idade + tempo_emprego + qt_pessoas_residencia', df)

In [18]:
modelo = sm.OLS(y,X).fit()

modelo.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.348
Model:                            OLS   Adj. R-squared:                  0.347
Method:                 Least Squares   F-statistic:                     333.6
Date:                Sat, 23 Nov 2024   Prob (F-statistic):               0.00
Time:                        15:15:00   Log-Likelihood:                -16212.
No. Observations:               15000   AIC:                         3.247e+04
Df Residuals:                   14975   BIC:                         3.266e+04
Df Model:                          24                                         
Covariance Type:            nonrobust                                         
=====================================================================================================================
                                                        coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------------------
Intercept                                             6.6644      0.218     30.508      0.000       6.236       7.093
C(sexo)[T.M]                                          0.7947      0.014     57.710      0.000       0.768       0.822
posse_de_veiculo[T.True]                              0.0351      0.013      2.693      0.007       0.010       0.061
posse_de_imovel[T.True]                               0.0897      0.013      6.978      0.000       0.064       0.115
C(tipo_renda)[T.Bolsista]                             0.2075      0.238      0.870      0.384      -0.260       0.675
C(tipo_renda)[T.Empresário]                           0.1543      0.015     10.457      0.000       0.125       0.183
C(tipo_renda)[T.Pensionista]                         -0.1910      0.021     -9.123      0.000      -0.232      -0.150
C(tipo_renda)[T.Servidor público]                     0.0571      0.022      2.602      0.009       0.014       0.100
C(educacao)[T.Pós graduação]                          0.0549      0.151      0.363      0.716      -0.241       0.351
C(educacao)[T.Secundário]                            -0.0719      0.056     -1.278      0.201      -0.182       0.038
C(educacao)[T.Superior completo]                      0.0479      0.057      0.845      0.398      -0.063       0.159
C(educacao)[T.Superior incompleto]                   -0.1024      0.063     -1.614      0.107      -0.227       0.022
C(estado_civil)[T.Separado]                           0.2754      0.107      2.568      0.010       0.065       0.486
C(estado_civil)[T.Solteiro]                           0.2294      0.105      2.181      0.029       0.023       0.436
C(estado_civil)[T.União]                             -0.0382      0.023     -1.662      0.097      -0.083       0.007
C(estado_civil)[T.Viúvo]                              0.2855      0.108      2.637      0.008       0.073       0.498
C(tipo_residencia, Treatment(1))[T.Aluguel]           0.0312      0.052      0.600      0.548      -0.071       0.133
C(tipo_residencia, Treatment(1))[T.Com os pais]       0.0196      0.029      0.676      0.499      -0.037       0.076
C(tipo_residencia, Treatment(1))[T.Comunitário]      -0.0795      0.090     -0.884      0.377      -0.256       0.097
C(tipo_residencia, Treatment(1))[T.Estúdio]           0.0931      0.079      1.181      0.238      -0.061       0.248
C(tipo_residencia, Treatment(1))[T.Governamental]     0.0299      0.034      0.866      0.386      -0.038       0.097
qtd_filhos                                           -0.2364      0.104     -2.262      0.024      -0.441      -0.032
idade                                                 0.0051      0.001      6.769      0.000       0.004       0.007
tempo_emprego  

In [19]:
# Criando o modelo usando ' + str(indice_casa) + ' como Treatment
y, X = patsy.dmatrices('np.log(renda) ~ C(sexo) + posse_de_veiculo + posse_de_imovel + qtd_filhos + C(tipo_renda) + C(educacao) + C(estado_civil) + C(tipo_residencia, Treatment(' + str(indice_casa) + ')) + idade + tempo_emprego + qt_pessoas_residencia', df)

In [20]:
modelo_1 = sm.OLS(y,X).fit()

modelo_1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.348
Model:                            OLS   Adj. R-squared:                  0.347
Method:                 Least Squares   F-statistic:                     333.6
Date:                Sat, 23 Nov 2024   Prob (F-statistic):               0.00
Time:                        15:15:00   Log-Likelihood:                -16212.
No. Observations:               15000   AIC:                         3.247e+04
Df Residuals:                   14975   BIC:                         3.266e+04
Df Model:                          24                                         
Covariance Type:            nonrobust                                         
=====================================================================================================================
                                                        coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------------------
Intercept                                             6.6644      0.218     30.508      0.000       6.236       7.093
C(sexo)[T.M]                                          0.7947      0.014     57.710      0.000       0.768       0.822
posse_de_veiculo[T.True]                              0.0351      0.013      2.693      0.007       0.010       0.061
posse_de_imovel[T.True]                               0.0897      0.013      6.978      0.000       0.064       0.115
C(tipo_renda)[T.Bolsista]                             0.2075      0.238      0.870      0.384      -0.260       0.675
C(tipo_renda)[T.Empresário]                           0.1543      0.015     10.457      0.000       0.125       0.183
C(tipo_renda)[T.Pensionista]                         -0.1910      0.021     -9.123      0.000      -0.232      -0.150
C(tipo_renda)[T.Servidor público]                     0.0571      0.022      2.602      0.009       0.014       0.100
C(educacao)[T.Pós graduação]                          0.0549      0.151      0.363      0.716      -0.241       0.351
C(educacao)[T.Secundário]                            -0.0719      0.056     -1.278      0.201      -0.182       0.038
C(educacao)[T.Superior completo]                      0.0479      0.057      0.845      0.398      -0.063       0.159
C(educacao)[T.Superior incompleto]                   -0.1024      0.063     -1.614      0.107      -0.227       0.022
C(estado_civil)[T.Separado]                           0.2754      0.107      2.568      0.010       0.065       0.486
C(estado_civil)[T.Solteiro]                           0.2294      0.105      2.181      0.029       0.023       0.436
C(estado_civil)[T.União]                             -0.0382      0.023     -1.662      0.097      -0.083       0.007
C(estado_civil)[T.Viúvo]                              0.2855      0.108      2.637      0.008       0.073       0.498
C(tipo_residencia, Treatment(1))[T.Aluguel]           0.0312      0.052      0.600      0.548      -0.071       0.133
C(tipo_residencia, Treatment(1))[T.Com os pais]       0.0196      0.029      0.676      0.499      -0.037       0.076
C(tipo_residencia, Treatment(1))[T.Comunitário]      -0.0795      0.090     -0.884      0.377      -0.256       0.097
C(tipo_residencia, Treatment(1))[T.Estúdio]           0.0931      0.079      1.181      0.238      -0.061       0.248
C(tipo_residencia, Treatment(1))[T.Governamental]     0.0299      0.034      0.866      0.386      -0.038       0.097
qtd_filhos                                           -0.2364      0.104     -2.262      0.024      -0.441      -0.032
idade                                                 0.0051      0.001      6.769      0.000       0.004       0.007
tempo_emprego  

Analisando esse primeiro modelo, podemos verificar uma certa quantidade de informações que não são relevantes para explicar a variáção da variável resposta com as outras variáveis independentes. A variável que possui a menor relação com a variável resposta nesse primeiro momento é a classe tipo_residencia seguida pela classe educação.

In [42]:
#2) Retirando a classe tipo_residencia do modelo

filtro_1 = 'np.log(renda) ~ C(sexo) + posse_de_veiculo + posse_de_imovel + qtd_filhos + C(tipo_renda) + C(educacao) + C(estado_civil) + idade + tempo_emprego + qt_pessoas_residencia'
y, X = patsy.dmatrices( filtro_1,df)

In [44]:
modelo_2 = sm.OLS(y,X).fit()

modelo_2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.348
Model:                            OLS   Adj. R-squared:                  0.347
Method:                 Least Squares   F-statistic:                     421.3
Date:                Sat, 23 Nov 2024   Prob (F-statistic):               0.00
Time:                        15:17:21   Log-Likelihood:                -16214.
No. Observations:               15000   AIC:                         3.247e+04
Df Residuals:                   14980   BIC:                         3.262e+04
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
======================================================================================================
                                         coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------
Intercept                              6.6655      0.218     30.538      0.000       6.238       7.093
C(sexo)[T.M]                           0.7958      0.014     57.955      0.000       0.769       0.823
posse_de_veiculo[T.True]               0.0350      0.013      2.686      0.007       0.009       0.061
posse_de_imovel[T.True]                0.0870      0.013      6.886      0.000       0.062       0.112
C(tipo_renda)[T.Bolsista]              0.2075      0.238      0.870      0.384      -0.260       0.675
C(tipo_renda)[T.Empresário]            0.1547      0.015     10.502      0.000       0.126       0.184
C(tipo_renda)[T.Pensionista]          -0.1902      0.021     -9.093      0.000      -0.231      -0.149
C(tipo_renda)[T.Servidor público]      0.0580      0.022      2.645      0.008       0.015       0.101
C(educacao)[T.Pós graduação]           0.0553      0.151      0.367      0.714      -0.241       0.351
C(educacao)[T.Secundário]             -0.0696      0.056     -1.239      0.215      -0.180       0.041
C(educacao)[T.Superior completo]       0.0496      0.057      0.875      0.382      -0.061       0.161
C(educacao)[T.Superior incompleto]    -0.1005      0.063     -1.585      0.113      -0.225       0.024
C(estado_civil)[T.Separado]            0.2771      0.107      2.584      0.010       0.067       0.487
C(estado_civil)[T.Solteiro]            0.2317      0.105      2.203      0.028       0.026       0.438
C(estado_civil)[T.União]              -0.0380      0.023     -1.655      0.098      -0.083       0.007
C(estado_civil)[T.Viúvo]               0.2862      0.108      2.645      0.008       0.074       0.498
qtd_filhos                            -0.2378      0.104     -2.277      0.023      -0.443      -0.033
idade                                  0.0051      0.001      6.755      0.000       0.004       0.007
tempo_emprego                          0.0618      0.001     60.529      0.000       0.060       0.064
qt_pessoas_residencia                  0.2618      0.104      2.511      0.012       0.057       0.466
==============================================================================
Omnibus:                        0.704   Durbin-Watson:                   2.021
Prob(Omnibus):                  0.703   Jarque-Bera (JB):                0.674
Skew:                           0.009   Prob(JB):                        0.714
Kurtosis:                       3.028   Cond. No.                     2.50e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.5e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

Após a retirada da categoria tipo_residencia, não houve pratiacmente nenhuma mudança nos parametros da análise, o que comprova a baixíssima interferência dessa variável na variável resposta.

In [47]:
#3) Removendo as outras categorias de baixa correlação com a resposta
filtro_2 = 'np.log(renda) ~ C(sexo) + posse_de_veiculo + posse_de_imovel + qtd_filhos + C(tipo_renda) + C(estado_civil) + idade + tempo_emprego + qt_pessoas_residencia'
y, X = patsy.dmatrices( filtro_2,df)

In [49]:
modelo_3 = sm.OLS(y,X).fit()

modelo_3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.344
Model:                            OLS   Adj. R-squared:                  0.343
Method:                 Least Squares   F-statistic:                     523.9
Date:                Sat, 23 Nov 2024   Prob (F-statistic):               0.00
Time:                        15:23:05   Log-Likelihood:                -16262.
No. Observations:               15000   AIC:                         3.256e+04
Df Residuals:                   14984   BIC:                         3.268e+04
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
=====================================================================================================
                                        coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
Intercept                             6.6691      0.212     31.494      0.000       6.254       7.084
C(sexo)[T.M]                          0.7888      0.014     57.360      0.000       0.762       0.816
posse_de_veiculo[T.True]              0.0448      0.013      3.439      0.001       0.019       0.070
posse_de_imovel[T.True]               0.0899      0.013      7.104      0.000       0.065       0.115
C(tipo_renda)[T.Bolsista]             0.2932      0.239      1.227      0.220      -0.175       0.762
C(tipo_renda)[T.Empresário]           0.1652      0.015     11.228      0.000       0.136       0.194
C(tipo_renda)[T.Pensionista]         -0.1891      0.021     -9.021      0.000      -0.230      -0.148
C(tipo_renda)[T.Servidor público]     0.0777      0.022      3.547      0.000       0.035       0.121
C(estado_civil)[T.Separado]           0.2727      0.108      2.535      0.011       0.062       0.484
C(estado_civil)[T.Solteiro]           0.2255      0.105      2.138      0.033       0.019       0.432
C(estado_civil)[T.União]             -0.0419      0.023     -1.820      0.069      -0.087       0.003
C(estado_civil)[T.Viúvo]              0.2689      0.109      2.478      0.013       0.056       0.482
qtd_filhos                           -0.2260      0.105     -2.157      0.031      -0.431      -0.021
idade                                 0.0047      0.001      6.251      0.000       0.003       0.006
tempo_emprego                         0.0619      0.001     60.404      0.000       0.060       0.064
qt_pessoas_residencia                 0.2511      0.105      2.401      0.016       0.046       0.456
==============================================================================
Omnibus:                        1.186   Durbin-Watson:                   2.020
Prob(Omnibus):                  0.553   Jarque-Bera (JB):                1.158
Skew:                           0.012   Prob(JB):                        0.560
Kurtosis:                       3.036   Cond. No.                     2.48e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.48e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

Comparando os resultados apresentados entre o primeiro modelo e o último, podemos perceber uma leve mudança nos parametros R2, onde no primeiro o resultado encontrado foi 348 com todas as variaéis no modelo contra 344 do último modelo já com o filtros aplicado. Em relação ao AIC, o primeiro modelo apresentou um valor de 3.247 contra um valor de 3.256 no último, indicando um leve aumento do valor do último para o primeiro.
Como os valores analisádos são bastante